<h1 align=center> MARS WEB SCRAPING </h1>

In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import shutil
import pandas as pd
import time
import pymongo

from IPython.display import Image, display, HTML # for printing to console; 

h1_line = '–'*25

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path, headless=False)
browser = Browser("chrome", **executable_path, headless=False)
# browser.quit()

## Step 1 - Scraping
Application developmnet was performed in this Jupyter Notebook. 

<hr>

### Visit [Nasa's Mars News website](https://mars.nasa.gov/news/) & retrieve the latest article's title & text

In [2]:
# Start @ Nasa's Mars News Homepage

nasa_news_home = "https://mars.nasa.gov/news/"
browser.visit(nasa_news_home)

print(f'Now Safe to Run Next Cell\n {nasa_news_home}')

Now Safe to Run Next Cell
 https://mars.nasa.gov/news/


##### * warning * : wait for previous cell to complete

In [3]:
# goal: Parse news page to find title & paragraph of latest news article; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
#browser.quit()
print(soup.body.prettify)

<bound method Tag.prettify of <body id="news" style="">
<svg display="none" height="0" width="0" xmlns="http://www.w3.org/2000/svg">
<symbol height="30" id="circle_plus" viewbox="0 0 30 30" width="30">
<g fill-rule="evenodd" transform="translate(1 1)">
<circle cx="14" cy="14" fill="#fff" fill-opacity=".1" fill-rule="nonzero" r="14" stroke="inherit" stroke-width="1"></circle>
<path class="the_plus" d="m18.856 12.96v1.738h-4.004v3.938h-1.848v-3.938h-4.004v-1.738h4.004v-3.96h1.848v3.96z" fill="inherit" stroke-width="0"></path>
</g>
</symbol>
<symbol height="30" id="circle_arrow" viewbox="0 0 30 30" width="30" xmlns="http://www.w3.org/2000/svg">
<g transform="translate(1 1)"><circle cx="14" cy="14" fill="#fff" fill-opacity=".1" r="14" stroke="inherit" stroke-width="1"></circle>
<path class="the_arrow" d="m8.5 15.00025h7.984l-2.342 2.42c-.189.197-.189.518 0 .715l.684.717c.188.197.494.197.684 0l4.35-4.506c.188-.199.188-.52 0-.717l-4.322-4.48c-.189-.199-.496-.199-.684 0l-.684.716c-.189.197-.1

In [5]:
#page > div.grid_list_page.module.content_page > div > article > div > section > div > ul > li:nth-child(1) > div > div
#print(soup.div.find('article_teaser'))

# Take out the <div> of name and get its value
article_title = soup.find(attrs={'class': 'content_title'}).text.strip()

article_paragraph = soup.find(attrs={'class': 'article_teaser_body'}).text.strip()
print(f'{article_title} \n\n{article_paragraph}')

Six Things to Know About NASA's Opportunity Rover 

Opportunity's mission is complete. Here are highlights from its time on Mars.


<hr>

### Visit [The Jet Propulsion Laboratory's Homepage](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)& extract the featured image <br>

In [6]:
# Start @ JPL Homepage
# browser = Browser("chrome", **executable_path, headless=False)
jpl_home = "https://www.jpl.nasa.gov"
image_thumb_url = jpl_home + "/spaceimages/?search=&category=Mars"
browser.visit(image_thumb_url)

In [7]:
# On JPL Homepage, automate clicking on 'FULL IMAGE' button; 
time.sleep(3)
xpath='//*[@id="full_image"]'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [8]:
# On Full Image Page, automate clicking on 'more info' button; 
time.sleep(3)
xpath = '//*[@id="fancybox-lock"]/div/div[2]/div/div[1]/a[2]'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()
print(f'Now Safe to Run Next Cell\n{jpl_home}')

Now Safe to Run Next Cell
https://www.jpl.nasa.gov


##### * warning * : wait for previous cell to complete

In [9]:
# goal: Parse more info page to find image url; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# Close Browser & Print BS object; 
#browser.quit()

print(soup.body.prettify())

<body class="light_background logged_out mobile_menu" id="image_detail" style="">
 <!--[if lt IE 9]>
      <div class='browsehappy' style='font-size: 30px; color: white; position:absolute; top: 0; margin: 0; height: 3000px; width: 100%; background: #000; z-index: 10000; padding: 5%;'>
        You are using an
        <strong>outdated</strong>
        browser. Please
        <a href='http://browsehappy.com/'>click here</a>
        to upgrade or change your browser.
      </div>
    <![endif]-->
 <div id="main_container">
  <div id="site_body">
   <div class="site_header_area">
    <header class="site_header">
     <div class="brand_area">
      <div class="brand1">
       <a class="nasa_logo" href="http://www.nasa.gov" title="NASA">
        NASA
       </a>
      </div>
      <div class="brand2">
       <div class="jpl_logo">
        <a href="//www.jpl.nasa.gov/" id="jpl_logo" title="Jet Propulsion Laboratory">
         Jet Propulsion Laboratory
        </a>
       </div>
       <div cl

In [11]:
#HTML ABOVE INSPECTED & NARROWED DOWN
#SELECTOR: page > section.content_page.module > div > article > figure > a
img_rel_path = soup.figure.a['href']
featured_img_url = jpl_home + img_rel_path
print(f'The url for the featured image in full size is: {featured_img_url}')

The url for the featured image in full size is: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18284_hires.jpg


In [12]:
# Download Copy of Image; 
img_path = 'current_files/jpl_fullsize_image.jpg'
response = requests.get(featured_img_url, stream=True)
with open(img_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
print(f'Image downloaded to: {img_path}')

Image downloaded to: current_files/jpl_fullsize_image.jpg


In [13]:
# Display Downloaded Image
Image(url='current_files/jpl_fullsize_image.jpg')

<hr>

### Retrieve latest tweet from the [Mars Weather Report's feed](https://twitter.com/marswxreport)<br>

In [14]:
# Start @ Twitterfeed
# browser = Browser("chrome", **executable_path, headless=False)
mwr_twitter_home = "https://twitter.com/marswxreport"
browser.visit(mwr_twitter_home)
print(f'Now Safe to Run Next Cell\n {mwr_twitter_home}')

Now Safe to Run Next Cell
 https://twitter.com/marswxreport


##### * warning * : wait for previous cell to complete

###### Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

In [15]:
# goal: Parse html to pinpoint contents of latest tweet; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [16]:
# Close Browser & Print BS object; 
#browser.quit()
print(soup.body.prettify())

<body class="three-col logged-out user-style-MarsWxReport enhanced-mini-profile ProfilePage ProfilePage--withWarning no-nav-banners supports-drag-and-drop dropdown-enabled" data-fouc-class-names="swift-loading no-nav-banners" dir="ltr">
 <div class="visuallyhidden" id="kb-shortcuts-msg">
  <h2>
   Keyboard Shortcuts
  </h2>
  <p>
   Keyboard shortcuts are available for common actions and site navigation.
   <button id="show-shortcuts-btn" tabindex="-1" type="button">
    View Keyboard Shortcuts
   </button>
   <button id="dismiss-shortcuts-btn" tabindex="-1" type="button">
    Dismiss this message
   </button>
  </p>
 </div>
 <script id="swift_loading_indicator" nonce="">
  document.body.className=document.body.className+" "+document.body.getAttribute("data-fouc-class-names");
 </script>
 <noscript>
  &lt;form action="https://mobile.twitter.com/i/nojs_router?path=%2Fmarswxreport" method="POST" class="NoScriptForm"&gt;
        &lt;input type="hidden" value="8e50c7d8050a2a734e998d45f8098

In [17]:
#HTML ABOVE INSPECTED & NARROWED DOWN
mars_weather = soup.find(attrs={'class': 'TweetTextSize'}).text.strip()
print(mars_weather)

#Opportunity the robot achieved so much because of #Opportunity the team.  People like @PlanetaryKeri, @doug_ellison, @marsroverdriver, @tanyaofmars,  @marssciencegrad, @cirquelar, @bellutta, @Dr_ThomasZ and so many more. Read more at #WRALhttps://www.wral.com/my-battery-is-low-and-it-s-getting-dark-end-of-opportunity-mission-a-loss-for-team/18197530/ …


In [18]:
mars_weather = mars_weather.split(sep='pic.twitter.com')[0]
print(f'the latest tweet: \n{mars_weather}')

the latest tweet: 
#Opportunity the robot achieved so much because of #Opportunity the team.  People like @PlanetaryKeri, @doug_ellison, @marsroverdriver, @tanyaofmars,  @marssciencegrad, @cirquelar, @bellutta, @Dr_ThomasZ and so many more. Read more at #WRALhttps://www.wral.com/my-battery-is-low-and-it-s-getting-dark-end-of-opportunity-mission-a-loss-for-team/18197530/ …


<hr>

### Retrieve the complete Mars table from the [Space Facts website](http://space-facts.com/mars/)<br>

In [19]:
# Start @ Space Facts Website
# browser = Browser("chrome", **executable_path, headless=False)
space_facts_home = "https://space-facts.com/mars/"
browser.visit(space_facts_home)
print(f'Now Safe to Run Next Cell\n {space_facts_home}')

Now Safe to Run Next Cell
 https://space-facts.com/mars/


##### * warning * : wait for previous cell to complete

In [20]:
# goal: use pandas to Parse & extract table; 
html = browser.html
mars_table_df = pd.read_html(html)
#browser.quit()
mars_table_df = mars_table_df[0]

In [21]:
mars_table_df.columns = ['Attribute', 'Details']
mars_table_df.set_index('Attribute', inplace=True)
mars_table_df[:][:]

,Details
Attribute,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [22]:
mars_table_df.columns

Index(['Details'], dtype='object')

In [23]:
mars_table_df.to_html('current_files/mars_table.html'.replace('\n', '')) #save df to html file & remove new line character/

In [24]:
# Display Downloaded Image
from IPython.display import HTML
display(HTML(filename='current_files/mars_table.html'))


#!open table.html

,Details
Attribute,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [25]:
# save html to variable
mars_table_html = mars_table_df.to_html()
# soup_table = BeautifulSoup(mars_table_html)
# print(soup_table.prettify)

to_replace = '<thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Details</th>\n    </tr>\n    <tr>\n      <th>Attribute</th>\n      <th></th>\n    </tr>\n  </thead>'
replace_with = '<thead>\n <tr style="text-align: right;">\n <th>Attribute</th>\n <th>Details</th>\n </tr>\n </thead>'
mars_table_html = mars_table_html.replace(to_replace, replace_with)

print(mars_table_html)

<table border="1" class="dataframe">
  <thead>
 <tr style="text-align: right;">
 <th>Attribute</th>
 <th>Details</th>
 </tr>
 </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [26]:
mars_table_html

'<table border="1" class="dataframe">\n  <thead>\n <tr style="text-align: right;">\n <th>Attribute</th>\n <th>Details</th>\n </tr>\n </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

<hr>

### Retrieve images of Mars' four hemispheres from [the Astrogeology Science Center](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)<br>


#### The Hemispheres are:
    - Cerberus
    - Schiaparelli
    - Syrtis Major
    - Valles Marineris

<h3 align=center>Cerberus</h3>

In [27]:
# Start @ USGS homepage
# browser = Browser("chrome", **executable_path, headless=False)
asgeo_home = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(asgeo_home)

In [28]:
# On automate clicking on 'Cereberus' image; 
xpath='//*[@id="product-section"]/div[2]/div[1]/a/img'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [29]:
# goal: Parse html to pinpoint image url; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# go back to homepage (after copying html) & print html body; 
browser.back()
# print(soup.body.prettify())

In [30]:
rows = soup.findAll('a')[42:43]
rows

[<a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif" target="_blank">Original</a>]

In [31]:
cerberus_href = soup.find('a', text='Original').get('href')
cerberus_sample_href = soup.find('a', text='Sample').get('href')
cerberus_href, cerberus_sample_href

('http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg')

In [32]:
# Download Copy of Image; 
img_path = 'current_files/Cerberus_fullsize_image.tif'
response = requests.get(cerberus_href, stream=True)
with open(img_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
print(f'Image downloaded to: {img_path}')

Image downloaded to: current_files/Cerberus_fullsize_image.tif


<h3 align=center>Schiaparelli</h3>

In [33]:
# On automate clicking on 'Schiaparelli' image; 
xpath='//*[@id="product-section"]/div[2]/div[2]/a/img'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [34]:
# goal: Parse Schiaparelli html to pinpoint image url; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# go back to homepage (after copying html) & print html body; 
browser.back()
# print(soup.body.prettify())

In [35]:
rows = soup.findAll('a')[42:43]
rows

[<a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif" target="_blank">Original</a>]

In [36]:
Schiaparelli_href = soup.find('a', text='Original').get('href')
Schiaparelli_sample_href = soup.find('a', text='Sample').get('href')
Schiaparelli_href, Schiaparelli_sample_href

('http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg')

In [37]:
# Download Copy of Image; 
img_path = 'current_files/Schiaparelli_fullsize_image.tif'
response = requests.get(Schiaparelli_href, stream=True)
with open(img_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
print(f'Image downloaded to: {img_path}')

Image downloaded to: current_files/Schiaparelli_fullsize_image.tif


<h3 align=center>Syrtis Major</h3>

In [38]:
# On automate clicking on 'Syrtis Major' image; 
xpath='//*[@id="product-section"]/div[2]/div[3]/a/img'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [39]:
# goal: Parse Syrtis Major html to pinpoint image url; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# go back to homepage (after copying html) & print html body; 
browser.back()
# print(soup.body.prettify())

In [40]:
rows = soup.findAll('a')[42:43]
rows

[<a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif" target="_blank">Original</a>]

In [41]:
Syrtis_Major_href = soup.find('a', text='Original').get('href')
Syrtis_Major_sample_href = soup.find('a', text='Sample').get('href')
Syrtis_Major_href, Syrtis_Major_sample_href

('http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg')

In [42]:
# Download Copy of Image; 
img_path = 'current_files/Syrtis_Major_fullsize_image.tif'
response = requests.get(Syrtis_Major_href, stream=True)
with open(img_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
print(f'Image downloaded to: {img_path}')

Image downloaded to: current_files/Syrtis_Major_fullsize_image.tif


<h3 align=center>Valles Marineris</h3>

In [43]:
# On automate clicking on 'Valles Marineris' image; 
xpath='//*[@id="product-section"]/div[2]/div[4]/a/img'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [44]:
# goal: Parse Valles Marineris html to pinpoint image url; 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# print html body; 
# print(soup.body.prettify())

In [45]:
soup.findAll('a')[42:43]

[<a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif" target="_blank">Original</a>]

In [46]:
Valles_Marineris_href = soup.find('a', text='Original').get('href')
Valles_Marineris_sample_href = soup.find('a', text='Sample').get('href')

Valles_Marineris_href, Valles_Marineris_sample_href

('http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg')

In [47]:
# Download Copy of Image; 
img_path = 'current_files/Valles_Marineris_image.tif'
response = requests.get(Valles_Marineris_href, stream=True)
with open(img_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
print(f'Image downloaded to: {img_path}')

Image downloaded to: current_files/Valles_Marineris_image.tif


In [48]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": Valles_Marineris_href},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_href},
    {"title": "Schiaparelli Hemisphere", "img_url": Schiaparelli_href},
    {"title": "Syrtis Major Hemisphere", "img_url": Syrtis_Major_href},
]
browser.quit()

<hr>
<hr>

## Step 2 - MongoDB and Flask Application
Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was 
scraped from the URLs above.

Start by converting your Jupyter notebook into a Python script called scrape_mars.py with a function called 
scrape that will execute all of your scraping code from above and return one Python dictionary containing 
all of the scraped data.

Next, create a route called /scrape that will import your scrape_mars.py script and call your scrape function.

Store the return value in Mongo as a Python dictionary.
Create a root route / that will query your Mongo database and pass the mars data into an HTML template to 
display the data.

Create a template HTML file called index.html that will take the mars data dictionary and display all of 
the data in the appropriate HTML elements. Use the following as a guide for what the final product should look 
like, but feel free to create your own design.

<hr>
<hr>

In [49]:
all_mars_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": Valles_Marineris_href, 'sample_url':Valles_Marineris_sample_href},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_href, 'sample_url':cerberus_sample_href},
    {"title": "Schiaparelli Hemisphere", "img_url": Schiaparelli_href, 'sample_url':Schiaparelli_sample_href},
    {"title": "Syrtis Major Hemisphere", "img_url": Syrtis_Major_href, 'sample_url':Syrtis_Major_sample_href},
    {"article_title": article_title, "article_paragraph": article_paragraph},
    {"featured_img_url": featured_img_url},
    {"mars_weather": mars_weather},
    {"mars_table_html": mars_table_html}
                ]

In [50]:
for url in all_mars_urls:
    print(f'{url} \n\n {h1_line*4}\n')

{'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'} 

 ––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'} 

 ––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

{'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'} 

 ––––––––––––––––––––––––––––––––––

In [51]:
## Step 2 - MongoDB and Flask Application

In [52]:
# Initialize PyMongo to work with MongoDBs
conn = "mongodb://127.0.0.1:27017"
client = pymongo.MongoClient(conn)

In [53]:
db = client.mars_db
collection = db.mars_collection

In [54]:
db.drop

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_db'), 'drop')

In [55]:
i=0;
#
for url in all_mars_urls:
    i += 1;
    # Dictionary to be inserted into MongoDB
    post = url
    collection.insert_one(post)
    print(f'Document Inserterd ⟶ #{i}: {h1_line*3}\n\n {url}\n\n')
#    print(f'{h1_line*4}\n')
    # Insert dictionary into MongoDB as a document


Document Inserterd ⟶ #1: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', '_id': ObjectId('5c6a442059f0e6048cedbee7')}


Document Inserterd ⟶ #2: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', '_id': ObjectId('5c6a442059f0e6048cedbee8')}


Document Inserterd ⟶ #3: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/downlo

In [56]:
links = collection.find()
i=0;



for link in links:
    i += 1;
    print(f'Document Read ⟵ #{i}: {h1_line*3}\n\n {link}\n\n')


Document Read ⟵ #1: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'_id': ObjectId('5c6a442059f0e6048cedbee7'), 'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


Document Read ⟵ #2: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'_id': ObjectId('5c6a442059f0e6048cedbee8'), 'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'sample_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}


Document Read ⟵ #3: –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

 {'_id': ObjectId('5c6a442059f0e6048cedbee9'), 'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedi

Step 3 - Submission
To submit your work to BootCampSpot, create a new GitHub repository and upload the following:

The Jupyter Notebook containing the scraping code used.
Screenshots of your final application.


Use Pymongo for CRUD: Create, Read, Update, Delete applications for your database. 